In [16]:
import os
import requests
import pandas as pd
import re
import time
import json
import csv
import uuid
import apivideo

from sqlalchemy import create_engine, text
from datetime import datetime, timedelta
from openai import OpenAI
from urllib.parse import quote_plus
from dotenv import load_dotenv
from pprint import pprint
from apivideo.api import videos_api
from apivideo.model.too_many_requests import TooManyRequests
from apivideo.model.videos_list_response import VideosListResponse
from apivideo.model.bad_request import BadRequest

load_dotenv()
vad_lexicon_filepath = 'NRC_VAD_Lexicon.csv'

# Database connection parameters
db_user = "postgres.gukeqqpzhaignmhdduma"  # usually this for Supabase
db_password = os.getenv("SUPABASE_PW")
db_host = "aws-0-us-east-1.pooler.supabase.com"  # from your Supabase connection settings
db_name = "postgres"  # usually this for Supabase
# Create connection string - note the quoted password to handle special characters
connection_string = f"postgresql://{db_user}:{quote_plus(db_password)}@{db_host}:5432/{db_name}"

openai = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

engine = create_engine(connection_string)


In [17]:
class ApiVideoAuth:
    def __init__(self, api_key):
        self.api_key = api_key
        self.access_token = None
        self.refresh_token = None
        self.token_expiration = None
        self.base_url = "https://ws.api.video"
        self.csv_file = "video_tags.csv"
        self.existing_tags = self._load_existing_tags()

    def authenticate(self):
        url = f"{self.base_url}/auth/api-key"
        headers = {"Content-Type": "application/json"}
        data = {"apiKey": self.api_key}

        response = requests.post(url, json=data, headers=headers)
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data["access_token"]
            self.refresh_token = token_data["refresh_token"]
            self.token_expiration = time.time() + token_data["expires_in"]
        else:
            raise Exception(
                f"Failed to authenticate: {response.status_code} - {response.text}"
            )

    def refresh_access_token(self):
        url = f"{self.base_url}/auth/refresh"
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
        }
        data = {"refreshToken": self.refresh_token}

        response = requests.post(url, json=data, headers=headers)
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data["access_token"]
            self.refresh_token = token_data["refresh_token"]
            self.token_expiration = time.time() + token_data["expires_in"]
        else:
            raise Exception(
                f"Failed to refresh token: {response.status_code} - {response.text}"
            )

    def get_access_token(self):
        if not self.access_token or time.time() >= self.token_expiration:
            print("Token expired or not available, refreshing...")
            self.refresh_access_token()
        return self.access_token

    def _load_existing_tags(self):
        existing_tags = {}
        if os.path.isfile(self.csv_file):
            with open(self.csv_file, "r", newline="") as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  # Skip header
                for row in reader:
                    video_id, tag = row
                    if video_id not in existing_tags:
                        existing_tags[video_id] = set()
                    existing_tags[video_id].add(tag)
        return existing_tags

    def _save_tags_to_csv(self, video_id, tags):
        new_tags = False
        if video_id not in self.existing_tags:
            self.existing_tags[video_id] = set()

        for tag in tags:
            if tag not in self.existing_tags[video_id]:
                self.existing_tags[video_id].add(tag)
                new_tags = True
                with open(self.csv_file, "a", newline="") as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([video_id, tag])


    def _make_request(self, method, endpoint, data=None, params=None, files=None):
        url = f"{self.base_url}{endpoint}"
        headers = {"Authorization": f"Bearer {self.get_access_token()}"}

        if data and not files:
            headers["Content-Type"] = "application/json"
            response = requests.request(
                method, url, json=data, params=params, headers=headers
            )
        else:
            response = requests.request(
                method, url, data=data, params=params, files=files, headers=headers
            )

        if response.status_code in [200, 201, 204]:
            return response.json() if response.content else None
        else:
            raise Exception(
                f"API request failed: {response.status_code} - {response.text}"
            )

    # Video endpoints
    def list_videos(self, params=None):
        """
        Retrieves a list of videos from the API
        
        Args:
            params (dict, optional): Query parameters to filter the video list
            
        Returns:
            dict: Response from the API containing video data
        """
        return self._make_request("GET", "/videos", params=params)

    def create_video(self, data):
        return self._make_request("POST", "/videos", data=data)

    def get_video(self, video_id):
        return self._make_request("GET", f"/videos/{video_id}")

    def update_video(self, video_id, data):
        return self._make_request("PATCH", f"/videos/{video_id}", data=data)

    def delete_video(self, video_id):
        return self._make_request("DELETE", f"/videos/{video_id}")

    def upload_video(self, video_id, file_path):
        with open(file_path, "rb") as file:
            return self._make_request(
                "POST", f"/videos/{video_id}/source", files={"file": file}
            )

    # Live stream endpoints
    def create_live_stream(self, data):
        return self._make_request("POST", "/live-streams", data=data)

    def get_live_stream(self, live_stream_id):
        return self._make_request("GET", f"/live-streams/{live_stream_id}")

    def update_live_stream(self, live_stream_id, data):
        return self._make_request("PATCH", f"/live-streams/{live_stream_id}", data=data)

    def delete_live_stream(self, live_stream_id):
        return self._make_request("DELETE", f"/live-streams/{live_stream_id}")

    # Player endpoints
    def create_player(self, data):
        return self._make_request("POST", "/players", data=data)

    def get_player(self, player_id):
        return self._make_request("GET", f"/players/{player_id}")

    def update_player(self, player_id, data):
        return self._make_request("PATCH", f"/players/{player_id}", data=data)

    def delete_player(self, player_id):
        return self._make_request("DELETE", f"/players/{player_id}")

    # Captions endpoints
    def upload_caption(self, video_id, language, file_path):
        with open(file_path, "rb") as file:
            return self._make_request(
                "POST", f"/videos/{video_id}/captions/{language}", files={"file": file}
            )

    def get_caption(self, video_id, language):
        return self._make_request("GET", f"/videos/{video_id}/captions/{language}")

    def update_caption(self, video_id, language, data):
        return self._make_request(
            "PATCH", f"/videos/{video_id}/captions/{language}", data=data
        )

    def delete_caption(self, video_id, language):
        return self._make_request("DELETE", f"/videos/{video_id}/captions/{language}")

    # Chapters endpoints
    def upload_chapter(self, video_id, language, file_path):
        with open(file_path, "rb") as file:
            return self._make_request(
                "POST", f"/videos/{video_id}/chapters/{language}", files={"file": file}
            )

    def get_chapter(self, video_id, language):
        return self._make_request("GET", f"/videos/{video_id}/chapters/{language}")

    def delete_chapter(self, video_id, language):
        return self._make_request("DELETE", f"/videos/{video_id}/chapters/{language}")

    # Watermark endpoints
    def upload_watermark(self, file_path):
        with open(file_path, "rb") as file:
            return self._make_request("POST", "/watermarks", files={"file": file})

    def delete_watermark(self, watermark_id):
        return self._make_request("DELETE", f"/watermarks/{watermark_id}")

    # Analytics endpoints
    def get_video_analytics(self, video_id, params=None):
        return self._make_request("GET", f"/analytics/videos/{video_id}", params=params)

    def get_live_stream_analytics(self, live_stream_id, params=None):
        return self._make_request(
            "GET", f"/analytics/live-streams/{live_stream_id}", params=params
        )

    # Helper functions
    def get_all_videos_for_person(self, person_names):
        tags = person_names if isinstance(person_names, list) else [person_names]
        return self.list_videos(params={"tags": tags})


In [5]:
class VTTUtils:
    @staticmethod
    def parse_timestamp(timestamp):
        """
        Parses a timestamp string and returns the total number of seconds.
        
        Supported formats:
        - mm:ss.xxx
        - hh:mm:ss.xxx
        
        Args:
            timestamp (str): The timestamp string to parse.
            
        Returns:
            float: Total seconds represented by the timestamp.
            
        Raises:
            ValueError: If the timestamp format is invalid.
        """
        parts = str(timestamp).split(':')
        
        if len(parts) == 2:
            minutes, seconds = parts
            hours = 0
        elif len(parts) == 3:
            hours, minutes, seconds = parts
        else: # Ghetto Fallback Mechanism
            seconds = 0
            minutes = 0 
            hours = 99
        
        try:
            total_seconds = int(hours) * 3600 + int(minutes) * 60 + float(seconds)
        except ValueError:
            raise ValueError("Invalid numerical values in timestamp.")
        
        return total_seconds

def extract_segments_by_ids(vtt_content, start_segment_id, end_segment_id):
    # Split the VTT content by double newlines to separate individual segments
    segments = vtt_content.strip().split("\n\n")
    
    # Initialize a list to hold relevant segments
    relevant_segments = []
    
    # Loop through each segment and process it
    for segment in segments:
        # Split the segment into lines (ID, timestamp, content)
        lines = segment.split("\n")
        
        # The first line is the segment ID, convert it to integer
        try:
            segment_id = int(lines[0].strip())
        except ValueError:
            # In case the first line is not a segment ID, skip this segment
            continue
        
        # Check if the segment ID is within the desired range
        if start_segment_id <= segment_id <= end_segment_id:
            relevant_segments.append(segment)
    
    # Join the relevant segments back together
    return "\n\n".join(relevant_segments)

def get_caption_text(video_id):
    api_video = ApiVideoAuth(os.getenv("API_VIDEO_API_KEY"))
    api_video.authenticate()

    openai = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    video = api_video.get_video(video_id)

    caption = api_video.get_caption(video["videoId"], "en")
    caption_url = caption['src']
    response = requests.get(caption_url)
    response.raise_for_status()
    caption_text = response.text
    time.sleep(3) # Sleep for 3 seconds

    return caption_text

def load_nrc_vad_lexicon(filepath):
    vad_lexicon = {}  # Initialize the dictionary
    df = pd.read_csv(filepath)
    for _, row in df.iterrows():
        word = row['word']
        vad_lexicon[word] = {
            'valence': 2 * row['valence'] - 1,  # Scaling to -1 to 1
            'arousal': 2 * row['arousal'] - 1,  # Scaling to -1 to 1
            'dominance': 2 * row['dominance'] - 1  # Scaling to -1 to 1
        }
    return vad_lexicon

def parse_vtt_to_df(content, video_id):
    blocks = re.split(r'\n\s*\n', content)
    data = []

    for block in blocks[1:]:  # Skip the WEBVTT and X-TIMESTAMP-MAP headers
        lines = block.strip().split('\n')
        if len(lines) >= 3:  # Ensure we have at least index, timing, and text
            index = int(lines[0])
            timing = lines[1]
            text = ' '.join(lines[2:])
            
              # Extract speaker from the text
            match = re.match(r'<v ([^>]+)>(.*)', text)
            if match:
                speaker, text = match.groups()
            else:
                speaker = ""
                
            # Extract start and end times
            start, end = timing.split(' --> ')

            data.append({
                'video_id': video_id, 
                'start': start,
                'end': end,
                'speaker': speaker,
                'text': text,
            })
    return pd.DataFrame(data)

def combine_consecutive_speakers(df):
        df['speaker_changed'] = df['speaker'] != df['speaker'].shift()
        df['group'] = df['speaker_changed'].cumsum()
        
        result = df.groupby('group').agg({
            'video_id': 'first',
            'start': 'first',
            'end': 'last',
            'speaker': 'first',
            'text': ' '.join
        }).reset_index(drop=True)
        
        result['index'] = range(0, len(result))
        result = result[['video_id','index','start','end','speaker','text']]
        return result

def calculate_duration(row):
    start = row['start'] 
    end = row['end']

    parts_start = start.split(':')
    if len(parts_start) == 2:
        minutes, seconds = parts_start
        hours = 0
    elif len(parts_start) == 3:
        hours, minutes, seconds = parts_start
    else:
        raise ValueError(f"Unexpected time format: {time_str}")

    seconds, milliseconds = seconds.split('.')
    
    total_seconds_start = (int(hours) * 3600 + int(minutes) * 60 + int(seconds) +
                     int(milliseconds) / 1000)

    parts_end = end.split(':')
    if len(parts_end) == 2:
        minutes, seconds = parts_end
        hours = 0
    elif len(parts_end) == 3:
        hours, minutes, seconds = parts_end
    else:
        raise ValueError(f"Unexpected time format: {time_str}")

    seconds, milliseconds = seconds.split('.')
    
    total_seconds_end = (int(hours) * 3600 + int(minutes) * 60 + int(seconds) +
                     int(milliseconds) / 1000)

    duration = total_seconds_end - total_seconds_start

    return duration

def count_words(text):
    words = re.findall(r"\b\w+\b", text.lower())
    return len(words)

def calculate_wpm(row):
    # Avoid division by zero
    if row['duration'] == 0:
        return 0
    # Convert duration to minutes and calculate WPM
    return (row['word_count'] / (row['duration'] / 60))

def comprehensive_text_analysis(text):
    # Word count
    words = re.findall(r"\b[a-z']+\b", text.lower())
    word_count = len(words)

    # Filler words and profanity
    hard_filler_words = set(['um', 'uh'])
    soft_filler_words = set(['like', 'you know', 'well', 'so', 'just', 
                        'kind of', 'sort of', 'i mean', 'basically', 'actually', 
                        'literally', 'honestly'])
    profanities = set(['damn', 'hell', 'shit', 'fuck', 'ass', 'bitch', 'bullshit'])

    hard_filler_count = sum(1 for word in words if word in hard_filler_words)
    soft_filler_count = sum(1 for word in words if word in soft_filler_words)
    profanity_count = sum(1 for word in words if word in profanities)

    # Question and sentence count
    question_count = text.count('?')
    sentence_count = len(re.findall(r'\w+[.!?]', text))

    return {
        'word_count': word_count,
        'hard_filler_count': hard_filler_count,
        'soft_filler_count': soft_filler_count,
        'profanity_count': profanity_count,
        'question_count': question_count,
        'sentence_count': sentence_count
    }

def calculate_vad_scores(text):
    words = re.findall(r"\b[a-z']+\b", text.lower())
    total_valence = total_arousal = total_dominance = word_count = 0

    for word in words:
        if word in vad_lexicon:
            scores = vad_lexicon[word]
            total_valence += scores['valence']
            total_arousal += scores['arousal']
            total_dominance += scores['dominance']
            word_count += 1

    # Calculate average VAD scores for the subtitle
    if word_count > 0:
        avg_valence = total_valence / word_count
        avg_arousal = total_arousal / word_count
        avg_dominance = total_dominance / word_count
    else:
        avg_valence = avg_arousal = avg_dominance = None  # No valid VAD words

    return {
        'avg_valence': avg_valence,
        'avg_arousal': avg_arousal,
        'avg_dominance': avg_dominance,
        'total_valence': total_valence,
        'total_arousal': total_arousal,
        'total_dominance': total_dominance,
        'vad_word_count': word_count
    }

def enrich_clean_vtt_df(df):
    df['duration'] = df.apply(calculate_duration, axis=1)
    
    # Apply the comprehensive text analysis function
    analysis_results = df['text'].apply(comprehensive_text_analysis)
    
    # Add new columns based on the analysis results
    for key in ['word_count', 'hard_filler_count', 'soft_filler_count', 
                'profanity_count', 'question_count', 'sentence_count']:
        df[key] = analysis_results.apply(lambda x: x[key])
    
    # Calculate VAD scores separately using the original 'text' column
    vad_results = df['text'].apply(calculate_vad_scores)
    
    # Add VAD-related columns
    for key in ['avg_valence', 'avg_arousal', 'avg_dominance', 
                'total_valence', 'total_arousal', 'total_dominance', 'vad_word_count']:
        df[key] = vad_results.apply(lambda x: x[key])
    
    return df

def build_clean_vtt(video_id):
    raw_vtt = get_caption_text(video_id)
    vtt_in_df = parse_vtt_to_df(raw_vtt, video_id)
    df = combine_consecutive_speakers(vtt_in_df)

    vtt_content = "WEBVTT\n\n"  # VTT header
    segment_id = 0
    for _, row in df.iterrows():
        # Format timestamp
        start_time = row['start']
        end_time = row['end']
        timestamp = f"{start_time} --> {end_time}"
        
        # Format speaker and text
        speaker = f"<v {row['speaker']}>"
        text = row['text']
        
        # Combine into VTT format
        vtt_content += f"{segment_id}\n{timestamp}\n{speaker} {text}\n\n"
        segment_id= segment_id + 1
    return vtt_content

def get_meeting_summary(clean, speaker):

    target_person = speaker
    caption_text = clean

    ## Adjusted prompt, based on requirement that 3 outputs are required
    prompt = f"""
        You are an expert in analyzing communication transcripts to summarize the content of a meeting. You need to provide three outputs, on three separate lines. 
        1) What type of meeting is this? (e.g. 1 on 1, team meeting, all-hands)
        2) Provide a two - three sententence summary of what the meeting is about. Use names and be specific as possible. Begin with "In this meeting"
        3) What role did {target_person} play in this meeting? Provide a two - three sententences
        OUPUT: 
        Meeting Type: ...
        Meeting Summary: In this meeting... 
        Target Role: In this meeting {target_person}... 
    """

    # Send prompt and caption text to OpenAI for processing
    chat_completion = openai.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"{prompt}\n\n{caption_text}",
            }
        ],
        model="gpt-4o-mini",
        temperature=0
    )

    # Extract response from the OpenAI completion
    meeting_summary = chat_completion.choices[0].message.content
    return meeting_summary

def get_vtt_df(video_id):
    raw_vtt = get_clean_vtt(video_id)
    vtt_in_df = parse_vtt_to_df(raw_vtt, video_id)
    df = combine_consecutive_speakers(vtt_in_df)
    return df

def build_clean_vtt_adjust(raw, video_id):
    raw_vtt = raw
    vtt_in_df = parse_vtt_to_df(raw_vtt, video_id)
    df = combine_consecutive_speakers(vtt_in_df)

    vtt_content = "WEBVTT\n\n"  # VTT header
    segment_id = 0
    for _, row in df.iterrows():
        # Format timestamp
        start_time = row['start']
        end_time = row['end']
        timestamp = f"{start_time} --> {end_time}"
        
        # Format speaker and text
        speaker = f"<v {row['speaker']}>"
        text = row['text']
        
        # Combine into VTT format
        vtt_content += f"{segment_id}\n{timestamp}\n{speaker} {text}\n\n"
        segment_id= segment_id + 1
    return vtt_content


##Prompts

In [20]:
# Variable Inputs for 
activity_max = "5"
sequence_max = "20"
sequence_min = "5"
activity_name = ""
prompt_mission = ""
agent_detection_prompt = ""
sublabel_prompt = ""

In [21]:
def feedback_prompt_builder(target_person):
    target_person = target_person
    global activity_name
    global prompt_mission
    global agent_detection_prompt
    global sublabel_prompt
    
    activity_name = "Feedback" 

    prompt_mission = """
        Your task is to identify / detect sequences of segments in the transcript where the participants give feedback.
        """

    agent_detection_prompt = f"""
        You are an expert linguist, whose job is to analyze communication transcripts. You will be provided a transcript in VTT format. It is a live recording of a meeting with multiple participants.
        
        Your task is to correctly assess if {target_person} is either giving feedback, recieving feedback, or not involved. Provide a rational for what you believe the answer is then answer giving feedback if {target_person} is the primary person giving feedback and answer recieving if you believe someone else is the primary person giving feedback. Answer with no additional information.

        Type Options and Definitions:
        - Giving Feedback: {target_person} is giving feedback to someone about something. This can include constructive feedback, positive reinforcement, guidence, or suggestions about ways to improve performance, a work product, teamm, or initiative.
        - Receiving Feedback: {target_person} is reciving feedback from someone about something. This can include constructive feedback, positive reinforcement, guidence, or suggestions about ways to improve performance, a work product, teamm, or initiative.
        - Not Involved: {target_person} is not involved in the interaction
        - Not Feedback: The interaction is not considered feedback at all.

        Example Output:
            {{
                "type": "Giving Feedback",
                "reasoning": "{target_person} is the primary person giving feedback in this case because they directly state 'I think you need to write out your thinking first then you can think more clearly' which indicates they are the primary actor in this transcript.",
            }}

        Do not explain yourself, do not deviate from the format, do not output additional data points.

        OUTPUT THE JSON OBJECT ONLY. Your output will be passed to `JSON.parse()`. Do not prefix with anything. Absolutely anything, not even ```json
        """

    sublabel_prompt = f"""
        You are an expert linguist, whose job is to analyze communication transcripts. World peace is at stake. 
        You will be provided a summary of a moment that occured within a meeting and then the transcript in VTT format from that moment. You will also be given the directionality of feedback in a JSON. It is a live recording of a meeting with multiple participants. 

        If the JSON input says 'Giving Feedback' then {target_person} is giving someone else feedback. If the JSON input is 'Receiving Feedback' then {target_person} is receiving feedback from someone else. Any other input means this is 'Not Feedback'

        Your task is to label each segment based on the category it most aligns with using the VTT transcript, and provide the result in JSON format with two fields: 'type' and 'reasoning'.

        Categories:
        1.	Positive Reinforcement: [Someone is affirming or encouraging anothers actions, behaviors, or performance to reinforce positive outcomes]
        2.	Constructive Feedback: [Someone is offering specific, actionable suggestions for improvement to something within the other persons control (e.g., work, performance)]
        3.	Critical Feedback: [Someone is pointing out negative or problematic behavior directly related to the person they are addressing.]
        4.	Guidance: [Someone is offering advice, support, or direction to help another solve a problem, improve, or grow.]
        5.	Suggestion: [Someone is proposing a new or alternative way of doing something, without directly offering criticism.]
        6.	Request: [Someone is asking or instructing another to complete a task, provide information, or take action.]
        7.	Other Feedback: [The communication does not fit into any of the feedback categories but is considered feedback]
        8. Not Feedback: [The communication does not fit into any of the feedback categories and is not considered feedback e.g., general conversation, unrelated comments)]

        Example Output:
            {{
                "type": "Positive Reinforcement",
                "reasoning": "Person1 offered positive feedback Person2: 'I like what you did there' during the transcript.",
            }}

        OUTPUT THE JSON OBJECT ONLY. Your output will be passed to `JSON.parse()`. Do not prefix with anything. Absolutely anything, not even ```json
        """
    set_templates()





In [22]:
def delegation_prompt_builder(target_person):
    target_person = target_person
    global activity_name
    global prompt_mission
    global agent_detection_prompt
    global sublabel_prompt

    activity_name = "Delegation" 

    prompt_mission = """
        Your task is to identify / detect sequences of segments in the transcript where the participants delegate to eachother.

        Some potential signs to look for in the transcript to identify delegation include but are not limited to:
	        1.	Task Assignment: One person directs another to complete a task (e.g., “Can you handle this?”).
	        2.	Authority Transfer: Responsibility or decision-making power is given (e.g., “You can decide on this”).
	        3.	Accountability: The delegatee is made responsible for the outcome (e.g., “I am counting on you for this”).
	        4.	Support Offered: Guidance or resources may be provided (e.g., “Let me know if you need help”).
	        5.	Timeline: Deadlines or expectations are set (e.g., “Complete this by Friday”).
        """

    agent_detection_prompt = f"""
        You are an expert linguist, analyzing communication transcripts. You will be provided a transcript in VTT format from a live meeting with multiple participants. Your task is to assess if {target_person} is delegating, receiving a delegated task, or not involved. Provide reasoning for your assessment and answer with delegating if {target_person} is the one delegating the task, receiving if they are being delegated a task, or no delegation if neither applies. Answer with no additional information.

        Type Options and Definitions:
        •	Delegating: {target_person} is delegating something to another person.
        •	Receiving: {target_person} is being delegated to by someone else.
        •	Not Involved: {target_person} is not involved in the interaction.
        •	No Delegation: The interaction does not involve delegation.

        Example Output:
            {{
                "type": "Delegating",
                "reasoning": "{target_person} is the primary person delegating because they say 'Can you handle this by Friday?' indicating task assignment."
            }}

        Do not explain yourself, do not deviate from the format, do not output additional data points.

        OUTPUT THE JSON OBJECT ONLY. Your output will be passed to `JSON.parse()`. Do not prefix with anything. Absolutely anything, not even ```json
        """

    sublabel_prompt = f"""
        You are an expert linguist, analyzing communication transcripts. World peace is at stake. You will be provided a summary of a moment that occurred within a meeting and then the transcript in VTT format from that moment. You will also be given the directionality of delegation in a JSON. It is a live recording of a meeting with multiple participants.

        If the JSON input says ‘Delegating’ then {target_person} is delegating a task or responsibility to someone else. If the input says ‘Receiving’ then {target_person} is being assigned a task or responsibility. Any other input means this is ‘Not Delegation’ 

        Your task is to label each segment based on the category it most aligns with using the VTT transcript, and provide the result in JSON format with two fields: ‘type’ and ‘reasoning’

        Categories:

            1.	Task Assignment: [Someone is assigning a specific task or responsibility to another person.]
            2.	Guidance with Delegation: [Someone is offering advice or support along with assigning a task to help the person succeed.]
            3.	Request for Action: [Someone is asking another person to take action or provide something, but not explicitly assigning a task.]
            4.	Authority Transfer: [Someone is giving another person decision-making authority over a specific task or responsibility.]
            5.	Receiving Delegation: [Someone is receiving a task, responsibility, or authority from another person.]
            6.	Other Delegation: [The communication involves delegation but does not clearly fit into the above categories.]
            7.	Not Delegation: [The communication does not involve delegation, e.g., unrelated conversation or general discussion.]

        Example Output:
            {{
                "type": "Task Assignment",
                "reasoning": "{target_person} is delegating by saying, 'Can you handle this report by tomorrow?' which assigns a task."
            }}

        OUTPUT THE JSON OBJECT ONLY. Your output will be passed to `JSON.parse()`. Do not prefix with anything. Absolutely anything, not even ```json
        """
    set_templates()

In [23]:
def decision_making_prompt_builder(target_person):
    target_person =target_person
    global activity_name
    global prompt_mission
    global agent_detection_prompt
    global sublabel_prompt

    activity_name = "Decision Making" 

    prompt_mission = """
       Your task is to identify / detect sequences of segments in the transcript where the participants are attempting to to make a decision, either successful or unsuccessful.
       Some potential signs to look for in the transcript to identify delegation include but are not limited to:
        1.	Problem or Opportunity Identification: A challenge or opportunity is raised, based on external or internal factors, framed around strategic goals.
        2.	Framing and Context Setting: The issue is outlined with relevant data, risks, and timelines to provide clarity for the decision-making process.
        3.	Discussion and Input Gathering: Input is gathered from team members, with a focus on options, priorities, and strategic alignment.
        4.	Options Evaluation: The team evaluates different paths, weighing risks, costs, and strategic fit, possibly facing competing priorities.
        5.	Decision Ownership: One individual (e.g., CEO or leader) takes responsibility for making the final call, based on the input gathered.
        6.	Consensus or Alignment Building: The decision-maker works to get team alignment, ensuring key stakeholders support the decision, whether this succeeds or not.
        7.	Actionable Decision: A decision is made, with clear steps and ownership assigned for execution. Success or failure depends on team alignment and execution..
        """

    agent_detection_prompt = f"""
        You are an expert linguist, analyzing communication transcripts. You will be provided a transcript in VTT format from a live meeting with multiple participants. Your task is to assess if {target_person} is participating in the decision-making process or not. Provide reasoning for your assessment and answer with Participating if {target_person} is actively engaged in the decision-making process or Not Participating if they are not involved. Answer with no additional information.

        Type Options and Definitions:

            •	Participating: {target_person} is actively contributing to the decision-making process by giving input, discussing options, or influencing the outcome.
            •	Not Participating: {target_person} is not involved in the decision-making process during this interaction.


            Example Output:
                {{
                    "type": "Participating",
                    "reasoning": "{target_person} provided input on the options being discussed, contributing to the decision-making."
                }}

            Do not explain yourself, do not deviate from the format, do not output additional data points.

            OUTPUT THE JSON OBJECT ONLY. Your output will be passed to `JSON.parse()`. Do not prefix with anything. Absolutely anything, not even ```json
        """

    sublabel_prompt = f"""
        You are an expert linguist, analyzing communication transcripts. World peace is at stake. You will be provided a summary of a moment that occurred within a meeting and then the transcript in VTT format from that moment. You will also be given the decision-making context in a JSON input. It is a live recording of a meeting with multiple participants.

        Your task is to label each segment based on the category it most aligns with using the VTT transcript and provide the result in JSON format with two fields: ‘type’ and ‘reasoning.’

        Categories:

            1.	Problem or Opportunity Identification: [Someone identifies a challenge or opportunity that needs attention, often based on external (market/customer) or internal (performance/resources) factors.]
            2.	Framing and Context Setting: [The issue is explained with data, risks, or timelines, helping the team understand the scope of the decision.]
            3.	Discussion and Input Gathering: [Input from multiple team members is sought, and the group discusses options, priorities, and trade-offs.]
            4.	Options Evaluation: [The team evaluates different paths, weighing risks, costs, and strategic alignment. Competing priorities may emerge.]
            5.	Decision Ownership: [One person takes responsibility for the final decision, based on input from the team.]
            6.	Consensus or Alignment Building: [The decision-maker works to get alignment from key stakeholders, ensuring the team is on board with the outcome.]
            7.	Actionable Decision: [A final, clear decision is made with steps, timelines, and ownership defined for execution.]
            8.	Not Decision-Making: [The communication does not involve decision-making, e.g., general conversation or unrelated comments.]

        Example Output:
            {{
                "type": "Task Assignment",
                "reasoning": "{target_person} is delegating by saying, 'Can you handle this report by tomorrow?' which assigns a task."
            }}
            
         OUTPUT THE JSON OBJECT ONLY. Your output will be passed to `JSON.parse()`. Do not prefix with anything. Absolutely anything, not even ```json
        """
    set_templates()

In [24]:
def motivation_prompt_builder(target_person):
    target_person = target_person
    global activity_name
    global prompt_mission
    global agent_detection_prompt
    global sublabel_prompt

    activity_name ="Dave"

    prompt_mission = """
    Your task is to identify / detect sequences of segments in the transcript where the participants are attempting to motivate or inpsire others. Look for these key indicators that someone is trying to motivate or inspire:

1. Vision Casting: Painting a compelling picture of future possibilities or positive outcomes, often using vivid language and emotional appeals to help others see what could be achieved.

2. Belief Reinforcement: Expressing confidence in others' abilities, highlighting their past successes, or acknowledging their potential to achieve goals.

3. Purpose Emphasis: Connecting tasks or goals to larger meaningful impacts, whether for the team, organization, customers, or society.

4. Growth Mindset Activation: Reframing challenges as learning opportunities, encouraging resilience, or emphasizing that abilities can be developed through effort.

5. Energy Building: Using dynamic language, enthusiasm, or call-and-response patterns to create positive emotional energy and collective momentum.

When reviewing the transcript, note that motivation attempts may combine multiple indicators and can range from brief encouraging statements to extended inspirational messages.
        """

    agent_detection_prompt = f"""
        You are an expert linguist, analyzing communication transcripts. You will be provided a transcript in VTT format from a live meeting with multiple participants. Your task is to assess if {target_person} is activly inspiring or motivating others. Provide reasoning for your assessment and answer with Participating if {target_person} is actively engaged in inspiring or motivating others process or Not Participating if they are not involved. Answer with no additional information.

        Type Options and Definitions:

            •	Participating: {target_person} is actively inspiring or motivating others.
            •	Not Participating: {target_person} is receiving inpiration or motivation from other, or not at all.


            Example Output:
                {{
                    "type": "Participating",
                      "reasoning": "{target_person} expressed confidence in others' abilities, highlighting their past successes, or acknowledging their potential to achieve goals to motivate and inspire"
                }}

            Do not explain yourself, do not deviate from the format, do not output additional data points.

            OUTPUT THE JSON OBJECT ONLY. Your output will be passed to `JSON.parse()`. Do not prefix with anything. Absolutely anything, not even ```json
        """

    sublabel_prompt = f"""
        You are an expert linguist, analyzing communication transcripts. World peace is at stake. You will be provided a summary of a moment that occurred within a meeting and then the transcript in VTT format from that moment. You will also be given the decision-making context in a JSON input. It is a live recording of a meeting with multiple participants.

        Your task is to label each segment based on the category it most aligns with using the VTT transcript and provide the result in JSON format with two fields: ‘type’ and ‘reasoning.’

        Categories:

          1. Vision Casting: [Speaker inspires through vivid descriptions of future possibilities or positive outcomes, using emotional appeals and compelling language to help others envision what could be achieved.]
          2. Belief Reinforcement: [Speaker builds confidence by explicitly acknowledging others' abilities, highlighting past successes, or emphasizing their potential to achieve goals.]
          3. Purpose Emphasis: [Speaker creates motivation by connecting immediate tasks or goals to larger meaningful impacts for the team, organization, customers, or society.]
          4. Growth Mindset Activation: [Speaker encourages development by reframing challenges as learning opportunities, fostering resilience, and emphasizing that abilities can be developed through effort.]
          5. Energy Building: [Speaker generates momentum through dynamic language, enthusiasm, or interactive patterns to create positive emotional energy and collective drive.]
          6. Not Motivational: [The communication does not involve attempts to motivate or inspire, e.g., purely informational or procedural discussion.]

        Example Output:
            {{
                "type": "Belief Reinforment",
                "reasoning": "{target_person} is motivating and inpsiring by saying, 'We got this, remmeber Q2, we also managed to hit our targets?' which highlights past sucesses."
            }}
            
         OUTPUT THE JSON OBJECT ONLY. Your output will be passed to `JSON.parse()`. Do not prefix with anything. Absolutely anything, not even ```json
        """
    set_templates()

In [25]:
json_data_scheme = ""
activity_detection_prompt = ""

vtt_data_structure_example = """
    {segment_number}
    {start_time} --> {end_time}
    <v {speaker}>{transcription_content}
    """

def set_templates():
  global json_data_scheme
  global activity_detection_prompt
  json_data_scheme = """
  {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "Sequence Schema",
    "type": "array",
    "items": {
      "type": "object",
      "properties": {
        "sequence_id": {
          "type": "integer",
          "description": "A unique identifier for each {activity_name} sequence"
        },
        "segment_id_sequence_start": {
          "type": "integer",
          "description": "The Segment number of the first segment within the identified {activity_name} sequence"
        },
        "segment_id_sequence_end": {
          "type": "integer",
          "description": "The Segment number of the last segment within the identified {activity_name} sequence"
        },
        "summary": {
          "type": "string",
          "description": "A brief explanation summarizing the interaction and why it was identified as {activity_name}"
        },
        "title": {
          "type": "string",
          "description": "A 4 to 6 word decsriptive for the clip that references the topic dicusussed"
        }
      },
      "required": [
        "sequence_id",
        "segment_id_sequence_start",
        "segment_id_sequence_end",
        "summary"
      ],
      "additionalProperties": false
    }
  }
  """


  # Full Prompt 
  activity_detection_prompt = f"""
      You are an expert linguist, whose job is to analyze communication transcripts. World peace is at stake.
      You will be provided a transcript in VTT format. It is a live recording of a meeting with multiple participants.The VTT contains an array of segments, each representing a portion of the conversation. Here is an example of one such segment, including a description of the key-value pairs:

      ```
      5
      00:24.494 --> 00:30.080
      <v William Hayden>And I just want to understand why that is what we can do to make sure that in the future

      6
      00:31.180 --> 00:36.340
      <v William Hayden>we retain top talent because that's like make or break for an organization, especially in this stage.
      ```

      This VTT transcript contains the following key components:
      ```
      Segment number: A unique identifier for this portion of the conversation
      Timestamp: The time range in which this segment of the conversation occurred
      Speaker: The name of the person speaking within angle brackets <v Speaker Name>
      Content: The actual text spoken during this segment
      ```

      ```
      {vtt_data_structure_example}
      ```
      
      The full transcript will consist of multiple such segments, each representing a distinct portion of the meeting conversation. These segments, when put together in order, form the complete transcript of the meeting.

      {prompt_mission}
      
      Those sequences must not overlap, meaning the same distinct segment must not be part of multiple sequences of segments where the {activity_name} has been identified / detected. Sequences should include context so someone reading it later can understand the interaction. Additionally, try to avoid very long sequences (no more than {sequence_max}) and very short sequences (no less than {sequence_min}); when the content of the transcript suggests a change of the conversation topic the sequence should end. Those sequences must not overlap.

      For each identified sequence of segments, provide the following data points in JSON format. Then, output your final result in a JSON-compatible array:

      JSON SCHEMA FOR THE OUTPUT:
      ```
  {json_data_scheme}
      ```


      Do not explain yourself, do not deviate from the format, do not output additional datapoints. Limit the number of outputs to a maximum of the {activity_max} most important instances of {activity_name}.

      OUTPUT THE JSON OBJECT ONLY. Your output will be passed to `JSON.parse()`. Do not prefix with anything. Absolutely anything, not even ```json
  """

In [3]:
def activity_detection(video_id):
    caption_text =  get_clean_vtt(video_id)

    # Send prompt and caption text to OpenAI for processing
    chat_completion = openai.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"{activity_detection_prompt}\n\n{caption_text}",
            }
        ],
        model="gpt-4o-mini",
        temperature=0,
    )

    # Extract response from the OpenAI completion
    activity_detection_response = chat_completion.choices[0].message.content

    # Initialize an empty DataFrame for storing enriched subtitles
    final_df = pd.DataFrame()
    pd.set_option("display.max_colwidth", None)

    # Assuming activity_detection_response is a JSON string or a list of dictionaries
    try:
        # Try loading it as JSON
        activity_data = pd.read_json(activity_detection_response)
    except ValueError:
        # If not a valid JSON string, try evaluating it as a list of dictionaries
        activity_data = pd.DataFrame(eval(activity_detection_response))

    # Create DataFrame with the activity detection data
    df_activity_detection = pd.DataFrame(activity_data)
    return df_activity_detection

def process_row(row):
    
    start_seq = row["segment_id_sequence_start"]
    end_seq = row["segment_id_sequence_end"]
    
    ## NEED TO FIX CURRENTLY TIMESTAMPS ARE WRONG
    vtt_df = get_vtt_df(video_id)
    
    try:
        start_timestamp = vtt_df.loc[vtt_df['index'] == start_seq, 'start'].values[0]
    except IndexError:
        print(f"No match found for start_seq {start_seq}")
        start_timestamp = None

    try:
        end_timestamp = vtt_df.loc[vtt_df['index'] == end_seq, 'end'].values[0]
    except IndexError:
        end_timestamp = vtt_df.loc[vtt_df['index'] == end_seq-1, 'end'].values[0]
    except IndexError:
        print(f"No match found for end_seq {end_seq}")
        end_timestamp = None


    return pd.Series(
        {
            "segment_start_timestamp": start_timestamp,
            "segment_end_timestamp": end_timestamp,
            "segment_start_timestamp_in_seconds": VTTUtils.parse_timestamp(start_timestamp),
            "segment_end_timestamp_in_seconds": VTTUtils.parse_timestamp(end_timestamp),
        }
    )


def process_activity_detection(df):
    df_activity_detection = df
    df_activity_detection[["segment_start_timestamp", "segment_end_timestamp", "segment_start_timestamp_in_seconds", "segment_end_timestamp_in_seconds"]] = (
        df_activity_detection.apply(process_row, axis=1)
    )
    return df_activity_detection


In [27]:
def finalize_activity_detection(df):
    final_df = df

    # Convert JSON strings in 'activity_analysis' and 'target_person_analysis' to dictionaries
    final_df['activity_analysis'] = final_df['activity_analysis'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    final_df['target_person_analysis'] = final_df['target_person_analysis'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

    # Extract 'type' and 'reasoning' from 'activity_analysis'
    final_df['activity_type'] = final_df['activity_analysis'].apply(lambda x: x['type'] if isinstance(x, dict) else None)
    final_df['activity_reasoning'] = final_df['activity_analysis'].apply(lambda x: x['reasoning'] if isinstance(x, dict) else None)

    # Extract 'type' and 'reasoning' from 'target_person_analysis' (treat as dictionary, not list)
    final_df['target_person_type'] = final_df['target_person_analysis'].apply(lambda x: x['type'] if isinstance(x, dict) else None)
    final_df['target_person_reasoning'] = final_df['target_person_analysis'].apply(lambda x: x['reasoning'] if isinstance(x, dict) else None)

    # Drop the original JSON columns
    final_df = final_df.drop(columns=['activity_analysis', 'target_person_analysis'])

    # Add a column that is always 'Feedback'
    final_df['activity'] = f"{activity_name}"

    # Convert segment_start_timestamp_in_seconds and segment_end_timestamp_in_seconds to integers
    final_df['segment_start_timestamp_in_seconds'] = final_df['segment_start_timestamp_in_seconds'].astype(int)
    final_df['segment_end_timestamp_in_seconds'] = final_df['segment_end_timestamp_in_seconds'].astype(int)

    # Add 'Moment_url' column based on video_id and segment_start_timestamp_in_seconds
    final_df['Moment_url'] = final_df.apply(lambda row: f"https://embed.api.video/vod/{row['video_id']}#;t={row['segment_start_timestamp_in_seconds']}", axis=1)

    return final_df

In [28]:
def get_video_idsv2():
    with apivideo.AuthenticatedApiClient(os.getenv("API_VIDEO_API_KEY")) as api_client:
        # Create an instance of the API class
        api_instance = videos_api.VideosApi(api_client)
        sort_by = "publishedAt" # str | Use this parameter to sort videos by the their created time, published time, updated time, or by title. (optional)
        sort_order = "asc" # str | Use this parameter to sort results. `asc` is ascending and sorts from A to Z. `desc` is descending and sorts from Z to A. (optional)
        page_size = 100 # int | Results per page. Allowed values 1-100, default is 25. (optional) if omitted the server will use the default value of 25

        # example passing only required values which don't have defaults set
        # and optional values
        try:
            # List all video objects
            api_response = api_instance.list(sort_by=sort_by, sort_order=sort_order, page_size=page_size)
            #pprint(api_response)
        except apivideo.ApiException as e:
            print("Exception when calling VideosApi->list: %s\n" % e)
        api_response = api_response.get('data')
        video_ids = [item['video_id'] for item in api_response]
        return video_ids

    return api_response

def get_video_object(video_id):
    # Enter a context with an instance of the API client
    with apivideo.AuthenticatedApiClient(os.getenv("API_VIDEO_API_KEY")) as api_client:
        # Create an instance of the API class
        api_instance = videos_api.VideosApi(api_client)
        video_id = video_id # str | The unique identifier for the video you want details about.

        # example passing only required values which don't have defaults set
        try:
            # Show a video
            api_response = api_instance.get(video_id)
        except apivideo.ApiException as e:
            print("Exception when calling VideosApi->get: %s\n" % e)
    return api_response

def get_delta(video_api_current_video_ids):
    all_video_ids = video_api_current_video_ids
    engine = create_engine(connection_string)
    # Run query and get results into a pandas DataFrame

    with engine.connect() as connection:
        query = text("SELECT video_api_id FROM public.meetings")
        result = connection.execute(query)
        df = pd.DataFrame(result.fetchall(), columns=result.keys())

    # Close connection
    engine.dispose()
    supabase_current_video_ids = list(df['video_api_id'])
    supabase_missing_video_ids = [video for video in all_video_ids if video not in supabase_current_video_ids]
    return supabase_missing_video_ids

def generate_meeting_table_data(supabase_missing_video_ids):
  
  if len(supabase_missing_video_ids) > 0:
    meetings = []
    # Create a list of dictionaries with the required fields
    for video_id in supabase_missing_video_ids:
        raw_data = get_video_object(video_id)
        original_vtt_file = get_caption_text(video_id)
        clean_vtt_file = build_clean_vtt_adjust(original_vtt_file, video_id)
        summary = get_meeting_summary(clean_vtt_file, video_id)
        video_data = {
        'video_api_id' : video_id,    
        'name' : raw_data.get('title'),
        'date' : raw_data.get('published_at'),
        'speaker' : raw_data.get('tags')[0],
        'original_vtt_file' : original_vtt_file,
        'clean_vtt_file' : clean_vtt_file,
        'summary' : summary
        }
        meetings.append(video_data)
    meetings = pd.DataFrame(meetings)
    return meetings

def write_meeting_table_data_to_supabase(data):
    meetings = data
    try:
        # Write DataFrame to PostgreSQL
        meetings.to_sql(
            'meetings',  # replace with your table name
            engine,
            if_exists='append',  # 'replace' if you want to overwrite, 'fail' if you want to error if exists
            index=False,
            method='multi',
            chunksize=1000)
        print("Data successfully written to database")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        engine.dispose()  # Clean up connection

def get_clean_vtt(video_id):
    engine = create_engine(connection_string)
    # Run query and get results into a pandas DataFrame

    with engine.connect() as connection:
        query = text("SELECT clean_vtt_file FROM public.meetings WHERE video_api_id = '" + video_id + "'" )
        result = connection.execute(query)
        df = pd.DataFrame(result.fetchall(), columns=result.keys())

    # Close connection
    engine.dispose()
    clean_vtt_file = df['clean_vtt_file']
    clean_vtt_file = clean_vtt_file[0]
    return clean_vtt_file

def update_segments(supabase_missing_video_ids):
  
  final_df_v2 = pd.DataFrame()
  for video_id in supabase_missing_video_ids:
    print(f"Processing video ID: {video_id}")
    raw_vtt = get_clean_vtt(video_id)
    vtt_in_df = parse_vtt_to_df(raw_vtt, video_id)
    clean_vtt_df = combine_consecutive_speakers(vtt_in_df)
    enriched_clean_vtt_df = enrich_clean_vtt_df(clean_vtt_df)

    final_df_v2 = pd.concat([final_df_v2, enriched_clean_vtt_df], ignore_index = True)
  
  print("VTT VAD dataset successfully build for all videos")
  
  # Your column mapping stays the same
  column_mapping = {
      'video_id': 'video_api_id',
      'start': 'start_timestamp',
      'end': 'end_timestamp',
      'speaker': 'speaker_name',
      'hard_filler_count': 	'hard_filler_word_count',
      'soft_filler_count': 'soft_filler_word_count', 	
  }
  
  final_df_v2_renamed = final_df_v2.rename(columns=column_mapping)
  
  # Create enhancement_id (two approaches you can try):
  # Approach 1 - direct UUID objects
  final_df_v2_renamed['id'] = final_df_v2_renamed.apply(lambda row: row['video_api_id'] + str(row['index']), axis=1) 
  
  segment_columns = ['id','video_api_id', 'index', 'start_timestamp', 
                    'end_timestamp', 'text', 'speaker_name', 'word_count', 'sentence_count', 'duration', 'hard_filler_word_count', 'soft_filler_word_count', 'profanity_count', 'question_count', 'vad_word_count', 'total_valence', 'total_arousal', 'total_dominance']
  
  segments = final_df_v2_renamed
  segments = segments[segment_columns]
  return segments

def write_segements_table_to_supabase(segments):
  # Write to database
  engine = create_engine(connection_string)
  try:
      # Write DataFrame to PostgreSQL
      segments.to_sql(
          'segments',  # replace with your table name
          engine,
          if_exists='append',  # 'replace' if you want to overwrite, 'fail' if you want to error if exists
          index=False,
          method='multi',
          chunksize=1000  # adjust based on your data size
      )
      print("Data successfully written to database")
  except Exception as e:
      print(f"An error occurred: {e}")
  finally:
      engine.dispose()  # Clean up connection


In [29]:
def add_new_moments(df):

  moments = df
  column_mapping = {
      'video_id': 'video_api_id',
      'Moment_url': 'moment_url' 	
  }
  
  moments_renamed = moments.rename(columns=column_mapping)
  
  # Create enhancement_id (two approaches you can try):
  # Approach 1 - direct UUID objects
  moments_renamed['id'] = [str(uuid.uuid4()) for _ in range(len(moments_renamed))]
  moments_renamed['latest'] = 'TRUE'
  moments_columns = ['id','segment_id_sequence_start', 'segment_id_sequence_end', 'summary', 'title', 'segment_start_timestamp', 'segment_end_timestamp', 'segment_start_timestamp_in_seconds', 'segment_end_timestamp_in_seconds', 'video_api_id', 'activity_type', 'activity_reasoning', 'target_person_type', 'target_person_reasoning', 'activity', 'moment_url', 'latest']
  
  
  moments_renamed = moments_renamed[moments_columns]
  
  # Write to database
  try:
      # Write DataFrame to PostgreSQL
      moments_renamed.to_sql(
          'moments',  # replace with your table name
          engine,
          if_exists='append',  # 'replace' if you want to overwrite, 'fail' if you want to error if exists
          index=False,
          method='multi',
          chunksize=1000  # adjust based on your data size
      )
      print("Data successfully written to database")
  except Exception as e:
      print(f"An error occurred: {e}")
  finally:
      engine.dispose()  # Clean up connection
  return moments_renamed

def expand_moments_to_segments(moments_df):
    """
    Expands a moments dataframe to create multiple rows for each sequence of segment IDs,
    where segment_id is a concatenation of video_api_id and the sequence number.
    
    Parameters:
    moments_df (pd.DataFrame): DataFrame with columns:
        - id
        - segment_id_sequence_start
        - segment_id_sequence_end
        - video_api_id
    
    Returns:
    pd.DataFrame: Expanded DataFrame with columns:
        - id (generated sequential ID)
        - moments_id (original moment ID)
        - segment_id (concatenated video_api_id + sequence number)
        - video_api_id
    """
    # Create empty list to store rows
    expanded_rows = []
    
    # Counter for new sequential IDs
    current_id = 1
    
    # Iterate through each moment
    for _, row in moments_df.iterrows():
        # Generate sequence of segment IDs
        segment_numbers = range(
            row['segment_id_sequence_start'],
            row['segment_id_sequence_end'] + 1
        )
        
        # Create a row for each segment ID
        for segment_number in segment_numbers:
            # Concatenate video_api_id with segment number
            segment_id = f"{row['video_api_id']}{segment_number}"
            
            expanded_rows.append({
                'id': str(uuid.uuid4()),
                'moments_id': row['id'],
                'segment_id': segment_id,
                'video_api_id': row['video_api_id']
            })
            current_id += 1
    moments_segment = pd.DataFrame(expanded_rows)
    # Create new DataFrame from expanded rows
    try:
    # Write DataFrame to PostgreSQL
      moments_segment.to_sql(
          'moments_segment',  # replace with your table name
          engine,
          if_exists='append',  # 'replace' if you want to overwrite, 'fail' if you want to error if exists
          index=False,
          method='multi',
          chunksize=1000  # adjust based on your data size
      )
      print("Data successfully written to database")
    except Exception as e:
      print(f"An error occurred: {e}")
    finally:
      engine.dispose()  # Clean up connection
    return moments_segment
  
def mark_old_latest_moment(videoids, activities):
   # Update query to set latest=FALSE
   query = text("UPDATE public.moments SET latest = FALSE WHERE video_api_id IN :videoids AND activity IN :activities AND latest = TRUE")
   print("Executing Query on Database: " + str(query))
   
   engine = create_engine(connection_string)
   
   # Run update query with parameters
   with engine.connect() as connection:
       result = connection.execute(query, {
           "videoids": tuple(videoids), 
           "activities": tuple(activities)
       })
       # Commit the transaction
       connection.commit()
   
   # Close connection
   engine.dispose()
   return result.rowcount  # Returns number of rows updated

In [30]:
vad_lexicon = load_nrc_vad_lexicon(vad_lexicon_filepath)
video_api_current_video_ids = get_video_idsv2()
supabase_missing_video_ids = get_delta(video_api_current_video_ids)
meetings = generate_meeting_table_data(supabase_missing_video_ids)
write_meeting_table_data_to_supabase(meetings)
segments = update_segments(supabase_missing_video_ids)
write_segements_table_to_supabase(segments)


Data successfully written to database
Processing video ID: vi5lUKF5SojsvO2eH5MdSXVs
Processing video ID: vi4iA7ftEHA7F7kXxBz5TW1s
Processing video ID: vi4Yb16priLky4Ze4x3ApG3C
Processing video ID: vi7GkawMp69I47lCmepGg2Y1
Processing video ID: vi17iPCCfSCPTTMMwSe8pF4v
Processing video ID: vi5oPyx5KnBrR7JX4o3IKRxL
Processing video ID: vi5tQVPfxGp8bEIsPrkYLc2t
Processing video ID: vimgkWn5yc9eZpob5nXB4k5
Processing video ID: vi82EUZk4sRGrV4MBYaYhyr
Processing video ID: vi4Ro4yE0PR9xSZ7JKYTU2r6
Processing video ID: vi4cHtqlN2qHZYWEcurYLXic
Processing video ID: vi2r3uY6mjoU6gc6QjaMcZFk
Processing video ID: vi2jqz22wk9sKKz8udDFa4Mv
Processing video ID: vi3lhBdLr1b48zymbStBq65m
Processing video ID: vi5RzuGHq6hVTXUZ6S2gc5FI
Processing video ID: vi1RUWYlkj8srU7xJWhvIgf4
Processing video ID: vi3veataLjXlJv4dEEvBGUIS
Processing video ID: vi5dVoW7a6py6gDqonevdQhG
Processing video ID: vi6izaRPNkne58IZ7zzjofeH
Processing video ID: vi4fEA572F2t8znnmHtDUsPu
VTT VAD dataset successfully build for all v

In [32]:
print(supabase_missing_video_ids)
supabase_missing_video_ids = ['vi5lUKF5SojsvO2eH5MdSXVs', 'vi4iA7ftEHA7F7kXxBz5TW1s', 'vi4Yb16priLky4Ze4x3ApG3C', 'vi7GkawMp69I47lCmepGg2Y1', 'vi17iPCCfSCPTTMMwSe8pF4v', 'vi5oPyx5KnBrR7JX4o3IKRxL', 'vi5tQVPfxGp8bEIsPrkYLc2t', 'vimgkWn5yc9eZpob5nXB4k5', 'vi82EUZk4sRGrV4MBYaYhyr', 'vi4Ro4yE0PR9xSZ7JKYTU2r6', 'vi4cHtqlN2qHZYWEcurYLXic', 'vi2r3uY6mjoU6gc6QjaMcZFk', 'vi2jqz22wk9sKKz8udDFa4Mv', 'vi3lhBdLr1b48zymbStBq65m', 'vi5RzuGHq6hVTXUZ6S2gc5FI', 'vi1RUWYlkj8srU7xJWhvIgf4', 'vi3veataLjXlJv4dEEvBGUIS', 'vi5dVoW7a6py6gDqonevdQhG', 'vi6izaRPNkne58IZ7zzjofeH', 'vi4fEA572F2t8znnmHtDUsPu']

['vi5lUKF5SojsvO2eH5MdSXVs', 'vi4iA7ftEHA7F7kXxBz5TW1s', 'vi4Yb16priLky4Ze4x3ApG3C', 'vi7GkawMp69I47lCmepGg2Y1', 'vi17iPCCfSCPTTMMwSe8pF4v', 'vi5oPyx5KnBrR7JX4o3IKRxL', 'vi5tQVPfxGp8bEIsPrkYLc2t', 'vimgkWn5yc9eZpob5nXB4k5', 'vi82EUZk4sRGrV4MBYaYhyr', 'vi4Ro4yE0PR9xSZ7JKYTU2r6', 'vi4cHtqlN2qHZYWEcurYLXic', 'vi2r3uY6mjoU6gc6QjaMcZFk', 'vi2jqz22wk9sKKz8udDFa4Mv', 'vi3lhBdLr1b48zymbStBq65m', 'vi5RzuGHq6hVTXUZ6S2gc5FI', 'vi1RUWYlkj8srU7xJWhvIgf4', 'vi3veataLjXlJv4dEEvBGUIS', 'vi5dVoW7a6py6gDqonevdQhG', 'vi6izaRPNkne58IZ7zzjofeH', 'vi4fEA572F2t8znnmHtDUsPu']


In [ ]:
activity_type_selector = [#"Feedback",
                         # "Decision Making",
                          "Delegation"#,
                         #"Motivation"
  #"Test"
                         ]

## Apply to meeting that have been recently added
#ideo_selector = filtered_data
video_selector = [{"videoId":"vi19hlTAbN1yu25fpWwdZI70", "tags":["Dave McGibbon"]}]
## Apply to meeting that have been recently added

#video_selector = ["vi21tWLEF2GadH1FxBaaUCVv"]
# Initialize ApiVideo and OpenAI instances
api_video = ApiVideoAuth(os.getenv("API_VIDEO_API_KEY"))
api_video.authenticate()

openai = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)


final_df = pd.DataFrame()

for video in video_selector:
    print("Starting Loop")
    target_person = video['tags']
    video_id = video['videoId']
    video = api_video.get_video(video_id)
    caption_text = build_clean_vtt(video_id)

    if "Motivation" in activity_type_selector:
        print(f"Processing video ID: {video_id}, for Motivation targeted towards {target_person}")

        motivation_prompt_builder(target_person)

        ## Get activity detection output from video 
        df_activity_detection = activity_detection(video_id)

        ## Add seconds timestamp to the segments
        df_activity_detection = process_activity_detection(df_activity_detection)

        ## Run sublabelling and agent detection 
        df_activity_detection = agent_detection_sublabeling(df_activity_detection)

        ## Format JSON output to columns
        final_df_iteration = finalize_activity_detection(df_activity_detection)

        ## Append results to final output df
        final_df = pd.concat([final_df, final_df_iteration], ignore_index = True)

    if "Feedback" in activity_type_selector:
        print(f"Processing video ID: {video_id}, for Feedback targeted towards {target_person}")

        feedback_prompt_builder(target_person)

        ## Get activity detection output from video 
        df_activity_detection = activity_detection(video_id)

        ## Add seconds timestamp to the segments
        df_activity_detection = process_activity_detection(df_activity_detection)

        ## Run sublabelling and agent detection 
        df_activity_detection = agent_detection_sublabeling(df_activity_detection)

        ## Format JSON output to columns
        final_df_iteration = finalize_activity_detection(df_activity_detection)

        ## Append results to final output df
        final_df = pd.concat([final_df, final_df_iteration], ignore_index = True)

    if "Delegation" in activity_type_selector:
        print(f"Processing video ID: {video_id}, for Delegation targeted towards {target_person}")

        delegation_prompt_builder(target_person)

        ## Get activity detection output from video 
        df_activity_detection = activity_detection(video_id)

        ## Add seconds timestamp to the segments
        df_activity_detection = process_activity_detection(df_activity_detection)

        ## Run sublabelling and agent detection 
        df_activity_detection = agent_detection_sublabeling(df_activity_detection)

        ## Format JSON output to columns
        final_df_iteration = finalize_activity_detection(df_activity_detection)

        ## Append results to final output df
        final_df = pd.concat([final_df, final_df_iteration], ignore_index = True)

    if "Decision Making" in activity_type_selector:
        print(f"Processing video ID: {video_id}, for Decision Making targeted towards {target_person}")

        decision_making_prompt_builder(target_person)

        ## Get activity detection output from video 
        df_activity_detection = activity_detection(video_id)

        ## Add seconds timestamp to the segments
        df_activity_detection = process_activity_detection(df_activity_detection)

        ## Run sublabelling and agent detection 
        df_activity_detection = agent_detection_sublabeling(df_activity_detection)

        ## Format JSON output to columns
        final_df_iteration = finalize_activity_detection(df_activity_detection)

        ## Append results to final output df
        final_df = pd.concat([final_df, final_df_iteration], ignore_index = True)

print("Video selection has been process for moments activities selection")
final_df.head(500)


In [ ]:
video_selector_only_ids = [item['videoId'] for item in video_selector] 

rows_updated = mark_old_latest_moment(video_selector_only_ids, activity_type_selector)
print(f"Found {rows_updated} old moments and set them to latest = False")
final_df = add_new_moments(final_df)
test  = (len(final_df))
print(f"{test} moments data has been added to the database")
print("Moments data has been added to the database")
debug = expand_moments_to_segments(final_df)
print("Moments Segement data has been added to the database")
debug.head()



In [31]:
## Test VideoID vi3MC0JsaO4n4cA7cYgYzpb4
## vi7hjMrHGRupQnnvk1PmunpJ
## vi4flB6clm0iow6IFTbC7emI
#filtered_data = update_meetings_table()
#if (len(filtered_data) > 0):
#  print("Building segments for " + str(len(filtered_data)) + "...")
#  # update_segments(filtered_data)
#else:
#  print("No new segments build...")